# Подбор гиперпараметров для RandomForest, CatBoost, XGBosst

In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline as mp
from sklearn.metrics import accuracy_score, f1_score, recall_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, cross_val_score
import optuna

Подготовка данных

In [2]:
train_df = pd.read_csv('../data/train_preprocessed.csv')
test_df = pd.read_csv('../data/test_preprocessed.csv')

X_train = train_df.drop('Churn', axis=1)
y_train = train_df['Churn']
X_test = test_df.drop('Churn', axis=1)
y_test = test_df['Churn']

scaler = StandardScaler()

pipeline = mp(
    scaler,
    SMOTE(random_state=42)
)

X_train_processed, y_train_processed = pipeline.fit_resample(X_train, y_train)

X_test_processed = scaler.transform(X_test)

C:\Users\илья\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] Не удается найти указанный файл
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\илья\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\subprocess.py", line 503, in run
    with Popen(*popenargs, **kwargs) as process:
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.

Использование RandomiseSearchCV для подбора гиперпараметров моделей

In [ ]:
scoring = ['accuracy', 'f1', 'recall', 'roc_auc']

param_desrtibutions = {
    'RF': {
        'n_estimators': np.random.randint(100, 500),
        'max_depth': np.random.randint(3, 20),
        'min_sample_split': np.random.randint(1, 20),
        'max_sample_split': np.random.randint(2, 20),
        'max_features': ['auto', 'sqrt', 'log2'],
        'criterion': ['gini', 'entropy', 'log_loss']
    },
    'CB': {
        'iterations': np.random.randint(100, 500),
        'depth': np.random.randint(4, 20),
        'learning_rate': np.random.uniform(0.4, 0.6),
        'l2_leaf_reg': np.random.randint(1, 10)
    },
    'XGB': {
        'n_estimators': np.random.randint(100, 500),
        'max_depth': np.random.randint(3, 20),
        'learning_rate': np.random.uniform(0.01, 0.3)
    }
}

models = {
    'RF': RandomForestClassifier(random_state=42),
    'CB': CatBoostClassifier(random_state=42, verbose=False),
    'XGB': XGBClassifier(random_state=42)
}

best_model ={}
seatch_results = {}

for name, model in models.items():
    random_search = RandomizedSearchCV(
        estimator=model,
        scoring=scoring,
        random_state=42,
        n_iter=20,
        n_jobs=-1,
        cv=5,
        refit='accuracy',
        return_train_score=True
    )

    random_search.fit(X_train_processed, y_train_processed)
    best_model[name] = random_search.best_estimator_
